[42, 70, 49, 46, 77, 63, 56, 81, 74, 67, 60, 53]
[46, 70, 74, 77, 53, 42, 63, 81, 67, 49, 60, 56]
[49, 77, 56, 42, 53, 70, 46, 63, 81, 74, 67, 60]
[53, 77, 81, 60, 46, 49, 70, 74, 56, 67, 42, 63]
[56, 63, 49, 60, 77, 53, 70, 42, 81, 74, 67, 46]
[60, 67, 53, 56, 77, 81, 63, 74, 46, 42, 49, 70]
[63, 70, 56, 67, 46, 60, 42, 77, 49, 81, 74, 53]
[67, 74, 60, 63, 70, 46, 81, 53, 42, 49, 56, 77]
[70, 46, 42, 77, 63, 74, 53, 67, 49, 56, 81, 60]
[74, 46, 81, 67, 70, 77, 53, 60, 42, 49, 56, 63]
[77, 53, 49, 70, 46, 81, 60, 74, 56, 42, 63, 67]
[81, 53, 74, 77, 60, 46, 67, 42, 49, 56, 63, 70]


True
{42: {42: 0.0, 70: 1.2649110640673518, 49: 1.2909944487358056, 46: 1.378404875209022}, 70: {42: 1.2649110640673518, 70: 0.0, 46: 1.224744871391589, 77: 1.2909944487358056, 63: 1.2909944487358056}, 49: {42: 1.2909944487358056, 49: 0.0, 77: 1.2649110640673518, 56: 1.2909944487358056}, 46: {42: 1.378404875209022, 70: 1.224744871391589, 46: 0.0, 74: 1.2649110640673518, 77: 1.2909944487358056}, 74: {46: 1.2649110640673518, 74: 0.0, 81: 1.2909944487358056, 67: 1.2909944487358056}, 77: {70: 1.2909944487358056, 49: 1.2649110640673518, 46: 1.2909944487358056, 77: 0.0, 53: 1.224744871391589, 81: 1.378404875209022}, 56: {49: 1.2909944487358056, 56: 0.0, 60: 1.378404875209022, 63: 1.2909944487358056}, 53: {77: 1.224744871391589, 53: 0.0, 81: 1.2649110640673518, 60: 1.2909944487358056}, 81: {74: 1.2909944487358056, 77: 1.378404875209022, 53: 1.2649110640673518, 81: 0.0}, 60: {56: 1.378404875209022, 53: 1.2909944487358056, 60: 0.0, 67: 1.2909944487358056}, 63: {70: 1.2909944487358056, 56: 1.290

0
1
0
1
1
0
1
1
1
1
1
1
0
0
0
0
1
0
1
0
1
0
0
0
1
0
1
1
0
0
0
1
0
1
0
0
0
1
0
1
0
1
1
1
0
1
1
0
1
0
0
1
0
1
0
1
0
0
1
1
0
1
0
1
0
1
0
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
1
0
1
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
1
1
0
0
1
1
1
1
0
1
0
0
1
1
1
1
0
0
1
1
1
1
0
0
0
1
1
1
0
1
1
0
0
0
1
0
1
1
1
0
0
1
1
1
0
0
0
1
0
1
1
0
0
0
0
0
1
1
1
1
0
0
1
1
1
0
0
0
1
0
1
1
0
1
0
0
1
1
0
1
1
0
1
0
0
1
0
1
0
1
1
0
1
0
1
0
0
1
1
1
1
0
0
0
0
1
0
0
1
1
0
1
1
0
0
1
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
1
1
0
0
1
0
0
0
0
1
1
0
0
1
1
1
1
1
1
1
0
1
1
1
1
0
0
1
1
0
0
1
0
0
1
0
0
1
1
1
0
0
1
1
1
0
1
1
1
0
0
1
1
1
0
0
0
1
0
0
1
0
0
1
0
0
1
1
0
1
1
0
0
1
1
1
1
1
0
1
1
1
1
1
0
0
1
0
1
1
1
1
1
0
1
0
0
1
0
1
1
1
1
1
1
1
0
0
0
1
1
0
1
1
0
0
0
1
1
0
1
1
0
0
0
1
1
0
0
0
0
1
1
0
0
0
1
1
1
0
1
0
1
1
1
0
1
1
0
0
0
1
1
1
1
0
1
1
0
1
0
0
0
0
0
0
1
1
0
1
1
0
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
1
0
1
1
0
0
0
1
0
0
0
1
1
0
0
0
0
1
0
1
1
1
1
0
1
0
1
1
0
1
0
0
1
1
1
1
0
1
1
1
1
1
0
0
0
1
1
1
1
0
1
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
0
1
0
1
1
0
0


In [331]:
import numpy as np


def writeYaml(filename,dictionary):    
    import yaml
    import io

    # Write YAML file
    with io.open(filename, 'w', encoding='utf8') as outfile:
        yaml.dump(dictionary, outfile, default_flow_style=False, allow_unicode=True)

def readYaml(filename):        
    # Read YAML file
    import yaml
    with open(filename, 'r') as stream:
        data_loaded = yaml.safe_load(stream)
    return data_loaded 

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)
    #return 2*gcd(a,b)**3/(a*b*(a+b))

G = Graph(loops=True)
KNN = 1+3
#idp = list(range(60-1*6,60+1*6+1))
idp = [42,46,49,53,56,60,63,67,70,74,77,81]
pitchsim = dict([])

def aa(n):
    #http://oeis.org/A018900
    b = 2
    i = int(np.floor((np.sqrt(8*n - 1) + 1)/2))
    j = n - 1 - i*(i - 1)/2
    return b**i + b**j

for p1 in idp:
    G.add_vertex(p1)
    sp = sorted([(kernPitch(p1,p2),p2) for p2 in idp ],reverse=True)
    perm = [x[1] for x in sp]
    pitchsim[p1] = perm
    print(perm)
    ss = sp[0:KNN]
    for t in ss:
        k,p = t
        G.add_edge(p1,p,float(np.sqrt(2*(1-k))))
plot(G).show()     
print(G.is_connected())
print(G.to_dictionary(edge_labels=True))

writeYaml("pitch_similarity.yaml",pitchsim)
writeYaml("simliarity_graph_3.yaml",dict(G.to_dictionary(edge_labels=True)))

def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause,tempo = inds[k][i]
            #print(pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(np.floor(pitch))
                if add21:
                    pitch += 21
                #print(pitch,times[k],duration,volume,isPause)    
                MyMIDI.addTempo(track,times[k], tempo)
                MyMIDI.addNote(track, channel, int(pitch), float(times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
            else:
                times[k] += duration*1.0
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  

def run_length_encoding(seq):
    compressed = []
    count = 1
    char = seq[0]
    for i in range(1,len(seq)):
        if seq[i] == char:
            count = count + 1
        else:
            compressed.append([char,count])
            char = seq[i]
            count = 1
    compressed.append([char,count])
    return compressed

def getComposition(s):
    v = log(s)/log(2)
    k = int(floor(v))
    d = Integer(s).digits(2,padto=k+1)
    rl = run_length_encoding(d)
    #print(t,d,rl)
    comp  = [x[1] for x in rl]
    return comp    

def getEncoding(s):
    v = log(s)/log(2)
    k = int(floor(v))
    d = Integer(s).digits(2,padto=k+1)
    return [i for i in range(len(d)) if d[i]==1]

def jacc(s1,s2):
    return len(s1.intersection(s2))/len(s1.union(s2))

def compositionSim(n1,n2):
    return jacc(set(getEncoding(n1)),set(getEncoding(n2)))


#compsim = dict([])
#for n1 in range(1,200):
#    ss = sorted([(compositionSim(n1,n2),-n2) for n2 in range(1,200)],reverse=True)
#    perm = [-x[1] for x in ss]
#    print(perm)
#    compsim[n1] = perm


def HH(start,end):
    H = Graph(loops=False)
    for a in range(start,end+1):
        H.add_vertex(a)
        for b in range(start,end+1):
            if abs(a-b) in [3,4]:
                H.add_edge(a,b)
    return H 

H = HH(16,96)
H.plot().show()

ccomps = [12*[1],6*[1],4*[1],3*[1],2*[1],1*[1]]

def numberToBase(x, b,numDigits=100):
    if RR(x) == 0:
        return [0]
    digits = []
    for n in range(numDigits):
        d = int(floor(x))%b
        digits.insert(0,d)
        x = b*(x-d)
    return digits[::-1]

pitchlistE = (numberToBase(exp(1),len(idp),numDigits=10000))


a3 = []
for n in range(1,2**len(ccomps)+1):
    if sum(Integer((n)).digits(2,padto=len(ccomps)))==3:
        a3.append(n)

a4 = []
for n in range(1,2**len(ccomps)+1):
    if sum(Integer((n)).digits(2,padto=len(ccomps)))==4:
        a4.append(n)      
        
def aa3(n):
    return a3[n-1]

def aa4(n):
    return a4[n-1]

iinds = []
seq = 2*[2]+4*[3]+8*[5]
comps = [44,2340] #50] #,52,54] #

numX = exp(1)
baseX = 2#len(idp)

#comps = [20,22,26] #,36,38,44,50,52,54]
vols = [32,48,64,64+16,96]
rndbits = [int(x)%2 for x in (str(pi.n(10000)))[2:]]
for r in range(len(ccomps)):
    sv = 50
    vols = []    
    noten = []
    lh=  []
    rh = []    
    durations = []
    for n in range(1,65):
        #cc = getComposition(comps[r])
        cc = ccomps[r]
        S = sum(cc)
        durations.extend([(c/S,n) for c in cc])
    k = 0   
    pitch = 60
    #ps = pitchsim[start][0:seq[(1)%len(seq)]]
    #ps.reverse() 
    #ps = pitchlistE[]
    v = 1
    sign = +1
    l = 0
    sd = 0
    while len(durations)>0:
        if v == len(vols)-1:
            sign = -1
        if v == 0:
            sign = +1
        vols.extend(H.shortest_path(sv,(sv+17)%(96-16)+16))
            #print(vols)
        sv = vols[-1]
        vol = vols.pop(0)        
        dur,num = durations.pop(0)
        sd+=dur
        octl = (num+r)%3
        octr = (num+r)%2
        digitX = int(floor(numX))%baseX
        print(digitX)
        numX = baseX*(numX-digitX)
        pitch = idp[(idp.index(pitch)+(-1)**digitX)%len(idp)]
        dd = Integer(num).digits(2,padto=len(ccomps))
        dd.reverse()
        pause = dd[r]==0
        tempo = 120 #80 if num <= 7 else 120 if num <=18 else 80 if num<=31 else 100 if num<=42 else 120
        noten.append((pitch,dur,vol,pause,tempo))
        nl = (pitch+3*octl,dur,vol-6*octl,pause,tempo)
        nr = (pitch-4*octr,dur,vol-6*octr,pause,tempo)
        lh.append(nl)
        rh.append(nr)                         
        v+=sign
        start = ps[-1]
        k+=1
        
        #if rndbits.pop(0)==1:
        
    iinds.append(noten) 
    #if r==1:
    #iinds.append(lh)                         
    #if r==2:
    #iinds.append(rh)                         
fn = "./midi/six_part_random_walk_on_euler_number_e.mid"    
writePitches(fn,iinds,tempo=120,instrument=(len(iinds))*[0],add21=False,start_at= len(iinds)*[0],durationsInQuarterNotes=False)        


In [271]:
for n in range(1,len(a4)+1):
    print(n,Integer(aa4(n)).digits(2,padto=len(ccomps)))

1 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
2 [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
3 [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
4 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
5 [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
6 [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
7 [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
8 [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
9 [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
10 [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
11 [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
12 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
13 [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
14 [0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
15 [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
16 [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
17 [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
18 [1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
19 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
20 [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
21 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
22 [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
23 [1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
24 [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
25 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
26 [1, 1,

In [334]:
def numberToBase(x, b,numDigits=100):
    if RR(x) == 0:
        return [0]
    digits = []
    for n in range(numDigits):
        d = int(floor(x))%b
        digits.insert(0,d)
        x = b*(x-d)
    return digits[::-1]

dd = (numberToBase(exp(1),2,numDigits=100))
ppoints = []

n = 0
for d in dd:
    n+=1
    if d==1:
        s +=1 
    if d==0:
        s -=1
        ppoints.append((n,s))
plot(ppoints).show()

ImportError: cannot import name 'OldScalarFormatter' from 'matplotlib.ticker' (/home/orges/.local/lib/python3.8/site-packages/matplotlib/ticker.py)